#Exercice : classe FichierTexte
Doria BONZI - M2 IdL - 20/12/2024

##Objectif

> On veut développer une classe FichierTexte permettant de manipuler des fichiers textes de différents formats : txt, WebAnno et Glozz. A minima, on veut pouvoir afficher le texte brut de tout fichier et compte le nombre de caractères et de mots.

##Fonctionnement de notre classe FichierTexte
Notre classe FichierTexte est la classe mère de trois classes filles :


*   Classe Txt
*   Classe WebAnno
*   Classe Glozz

Ces trois classes partagent 6 méthodes :


*   `lire` : lit le contenu du fichier
*   `afficher_texte_brut` (méthode abstraite car différente pour chaque classe fille) : récupère uniquement le texte "brut" du document (sans les annotations)
*   `compter_caracteres` : compte le nombre de caractères dans le texte brut du fichier
*   `compter_mots` : compte le nombre de mots dans le texte brut du fichier avec une tokenisation très sommaire
*   `convertir` (méthode abstraite car différente pour chaque classe fille) : convertit un fichier d'un format à l'autre
*   `afficher_annotations` (méthode abstraite car différente pour chaque classe fille) : crée un fichier HTML affichant le texte avec ses annotations



---



Actuellement, voici ce qu'il est possible de faire avec notre classe FichierTexte :



*   Pour la **classe Txt** : afficher le texte brut, compter les mots et les caractères; (pas d'annotations pour le format txt donc pas de `afficher_annotations` ni de `convertir`)
*   Pour la **classe WebAnno** : afficher le texte brut, compter les mots et les caractères, convertir vers un format txt uniquement, afficher les annotations dans une page Web (HTML).
*   Pour la **classe Glozz** : afficher le texte brut, compter les mots et les caractères, convertir vers un format txt uniquement, afficher les annotations dans une page Web (HTML).



> NB. Pour la méthode `afficher_annotations`, nous vous invitons à télécharger le fichier HTML généré dans le dossier /content/ de ce notebook, puis de l'ouvrir "en local" pour visualiser la page avec le texte et ses annotations.

> Pour la classe WebAnno, nous utilisons deux fichiers, un fichier avec des annotations POS et un fichier avec des annotations coréférences. Pour la classe Glozz, nous avons un groupe de fichiers concernant des annotations coréférences.


---


###Pistes d'améliorations

*   A l'heure actuelle, il est impossible de convertir un fichier WebAnno en fichier Glozz et inversement.
*   Il est aussi impossible de visualiser les annotations de type coréférences de manière "claire" en format WebAnno et Glozz.
*   Notre méthode `afficher_annotations` pour la classe Glozz ne semble récupérer aucune relation entre les unités.





#Classe FichierTexte

##Importations

In [ ]:
!pip install gdown

#Fichier format txt
!gdown --id 17O-8eGlNagqhmgDSj27emCctp1XUgaNM -O lorem.txt

#Fichier format Glozz
!gdown --id 1oqTKQNJ00X0nPahEYs7KUVYFbRx7rioa -O glozz_annot_test.aa #annotations, corrections orthographiques
!gdown --id 1BytMAEYT39YEdPD6zetJjCRtojngS8Az -O glozz_tags_test.aam #tags annotations
!gdown --id 1yRYd3O6RDFKgyuwVA8Yz1RikHqp8wnFs -O glozz_txt_test.ac #texte brut

#Fichier format WebAnno
!gdown --id 1zrvOBu9bwsv_AJUAEAI8EhBpWe3hKA6H -O EC-CE1-2015-S47_doc000001_webanno.tsv #annotations POS
!gdown --id 1eQPNyWB4vosy_kkPxC8nSE74OwLjZZNs -O essai.tsv #annotations coreferences

In [ ]:
from abc import ABC,abstractmethod
import pandas as pd
import webbrowser #pour ouvrir nos fichiers annotés dans le navigateur
import json
import xml.etree.ElementTree as ET #pour traiter le xml des fichiers glozz

##Classe mère et classes filles

###Classe FichierTexte

In [ ]:
class FichierTexte(ABC):
    def __init__(self, nom_fichier):
        self.nom_fichier=nom_fichier
        self.contenu=None

    def lire(self):
        """Lit le contenu du fichier"""
        with open(self.nom_fichier,'r',encoding='utf-8') as f:
            self.contenu=f.readlines()
        #print(self.contenu)
        return self.contenu

    @abstractmethod
    def afficher_texte_brut(self):
        """Affiche le texte brut"""
        pass

    def compter_caracteres(self):
        """Compte le nombre de caractères du texte sans annotations"""
        texte_brut=self.afficher_texte_brut()
        return len(texte_brut)

    def compter_mots(self):
        """Compte le nombre de mots du texte sans annotations"""
        texte_brut=self.afficher_texte_brut()
        return len(texte_brut.split()) #tokenisation très sommaire pour compter les mots du texte brut

    @abstractmethod
    def convertir(self, format_cible):
        """Convertit le fichier vers un autre format"""
        pass

    @abstractmethod
    def afficher_annotations(self):
        """Affiche les textes avec les annotations"""
        pass

###Classe Txt

In [ ]:
class Txt(FichierTexte):
    def afficher_texte_brut(self):
        """Affiche le texte brut"""
        if self.contenu is None:
            with open(self.nom_fichier,'r',encoding='utf-8') as f:
                self.contenu=f.read()  #lire tout le contenu comme une chaîne
        return self.contenu

    def convertir(self,format_cible):
        raise NotImplementedError("Conversion pour txt non implémentée.")

    def afficher_annotations(self):
        print("Pas d'annotations pour le format txt")

###Classe WebAnno

In [ ]:
class WebAnno(FichierTexte):
    def afficher_texte_brut(self):
        """
        Affiche uniquement le texte sans annotations
        """
        if self.contenu is None:
            self.lire()  #si la méthode "lire" n'a pas encore été utilisée, on lit le fichier
        #print(self.contenu)

        texte_brut="" #on stocke ici notre texte non annoté

        #extractation et affichage des parties de texte après "#Text=" dans la console
        for ligne in self.contenu:
            if ligne.startswith("#Text="):
                texte_brut+=ligne.strip()[6:]+"\n"  #on ajoute chaque texte récupéré à texte_brut

        return texte_brut

    def convertir(self,format_cible):
        """
        Convertit le fichier WebAnno TSV en texte brut ou format Glozz
        """
        if format_cible=="txt":
            texte_brut=self.afficher_texte_brut()
            with open(self.nom_fichier.replace(".tsv",".txt"), "w",encoding='utf-8') as f:
                f.write(texte_brut)
            print(f"Conversion vers texte brut réussie : {self.nom_fichier.replace('.tsv','.txt')}")


        elif format_cible=="glozz":
            raise NotImplementedError("Conversion pour txt non implémentée.")

        else:
            print(f"Format de conversion '{format_cible}' non supporté")


    def afficher_annotations(self):
        """
        Génère un fichier HTML pour afficher les textes annotés à partir de fichiers TSV
        """
        if self.contenu is None:
            self.lire()

        #ici on commence notre fichier html pour afficher les annotations
        html_content="""
        <!DOCTYPE html>
        <html lang="fr">
        <head>
            <meta charset="UTF-8">
            <title>Annotations WebAnno</title>
            <style>
                body{
                    font-family: Arial,sans-serif;
                    line-height:1.8;
                    padding:20px;
                    background:#eaeaea; /*gris clair*/
                }
                .phrase{
                    margin-bottom:20px;
                }
                .token{
                    display:inline-block;
                    margin-right:10px;
                    font-size:18px;
                    color:black;
                }
                .annotations{
                    display:flex;
                    flex-direction: column;
                    align-items:center;
                    font-size:14px;
                    margin-top:-10px;
                }
                .annotation{
                    color:grey;
                    font-style:italic;
                } /*on associe une couleur à chaque annotation*/
                .annotation:nth-child(1) {color:blue;}
                .annotation:nth-child(2) {color:green;}
                .annotation:nth-child(3) {color:orange;}
                .annotation:nth-child(4) {color:purple;}
                .annotation:nth-child(5) {color:brown;}
                .annotation:nth-child(6) {color:pink;}
                .annotation:nth-child(7) {color:gray;}
                .annotation:nth-child(8) {color:red;}
            </style>
        </head>
        <body>
            <h1 style="text-align: center;">Texte annoté</h1>
            <div style="background: #fff; padding: 20px; border: 1px solid #ddd; border-radius: 8px;">
        """

        #A présent, on traite nos données pour enrichir notre fichier HTML
        phrases=[]
        phrase=[]
        for ligne in self.contenu:
            if ligne.startswith("#Text="):  #on détecte une nouvelle phrase avec #Text
                if phrase:
                    phrases.append(phrase)
                phrase=[] #on vide la variable phrase une fois la phrase courante dans la liste "phrases"
            elif ligne.strip() and not ligne.startswith("#"): #si ça ne commence pas par #, on récup les infos
                colonnes=ligne.strip().split("\t")
                if len(colonnes)>3:  #on considère toutes les colonnes après la troisième comme des annotations
                    token=colonnes[2]
                    annotations=colonnes[3:]
                    phrase.append((token,annotations))
        if phrase: #ajout de la dernière phrase
            phrases.append(phrase)

        #maintenant, on alimente notre HTML
        for phrase in phrases:
            html_content+='<div class="phrase">\n'
            for token,annotations in phrase:
                html_content+=f"""
                    <div class="token">
                        {token}
                        <div class="annotations">
                """
                for annotation in annotations: #on ajoute les annotations au html
                    html_content+=f'<div class="annotation">{annotation}</div>'
                html_content+="""
                        </div>
                    </div>
                """
            html_content+='</div>\n'

        #fin du html
        html_content+="""
        </body>
        </html>
        """

        #on sauvegarde le fichier html produit
        nom_fichier_html=self.nom_fichier.replace(".tsv","_visualisation.html")
        with open(nom_fichier_html,"w",encoding="utf-8") as f:
            f.write(html_content)


        #on l'ouvre dans le navigateur
        #? marche pas // à voir en local
        print(f"Affichage dans le navigateur : {nom_fichier_html}")
        webbrowser.open(nom_fichier_html)

###Classe Glozz

In [ ]:
class Glozz(FichierTexte):
    def __init__(self,nom_fichier_ac,nom_fichier_aam,nom_fichier_aa):
        """
        Initialise la classe Glozz avec trois fichiers :
          - nom_fichier_ac : fichier texte brut (.ac)
          - nom_fichier_aam : fichier de modèle (.aam)
          - nom_fichier_aa : fichier d'annotations (.aa)
        """
        super().__init__(nom_fichier_ac)  #fichier ac traité comme le fichier "principal" pour texte brut
        self.nom_fichier_aam=nom_fichier_aam
        self.nom_fichier_ac = nom_fichier_ac
        self.nom_fichier_aa=nom_fichier_aa
        self.modele=None
        self.annotations=None

    def lire(self):
        """
        Lit le contenu des trois fichiers :
        - Texte brut depuis le fichier .ac
        - Modèle depuis le fichier .aam
        - Annotations depuis le fichier .aa
        """
        #lecture du fichier texte brut (.ac)
        with open(self.nom_fichier,'r',encoding='utf-8') as f:
            self.contenu=f.read()

        #lecture du fichier de métadonnées (.aam)
        try:
            with open(self.nom_fichier_aam,'r',encoding='utf-8') as f:
                self.modele=f.read()
        except FileNotFoundError:
            print(f"Le fichier {self.nom_fichier_aam} est introuvable.")
            self.modele=None

        #lecture du fichier d'annotations (.aa)
        try:
            with open(self.nom_fichier_aa,'r',encoding='utf-8') as f:
                self.annotations=json.load(f)  #on charge un json des annotations
        except json.JSONDecodeError:
            print(f"Le fichier {self.nom_fichier_aa} n'est pas un JSON valide.")
            self.annotations={}
        except FileNotFoundError:
            print(f"Le fichier {self.nom_fichier_aa} est introuvable")
            self.annotations=None

    def afficher_texte_brut(self):
        """
        Retourne uniquement le texte brut du fichier .ac
        """
        if self.contenu is None:
            self.lire()
        return self.contenu



    def convertir(self,format_cible):
        """
        Convertit le fichier vers un format txt ou WebAnno
        """
        if format_cible=="txt":
            texte_brut=self.afficher_texte_brut()
            with open(self.nom_fichier.replace(".ac",".txt"), "w",encoding='utf-8') as f:
                f.write(texte_brut)
            print(f"Conversion vers texte brut réussie : {self.nom_fichier.replace('.ac','.txt')}")


        elif format_cible=="WebAnno" or "webanno":
            raise NotImplementedError("Conversion vers WebAnno non implémentée.")

        else:
            print(f"Format de conversion '{format_cible}' non supporté")


    def afficher_annotations(self):
        """
        Génère une page HTML affichant le texte brut avec les annotations et relations.
        """
        if self.annotations is None:
            self.lire() #chargement des annotations

        #chargmeent du texte brut
        texte_brut=self.afficher_texte_brut()

        #on parse le fichier xml (.aa)
        try:
            tree=ET.parse(self.nom_fichier_aa)
            root=tree.getroot()
        except ET.ParseError as e:
            print(f"Erreur lors du parsage du fichier XML : {e}")
            return

        #on essaie de lier des "unités" définies par des id et des "types", à des relations définies par un id et deux "terms" (unités liées)
        unites={}
        relations=[]
        relation_data={}

        for element in root:
            if element.tag=="unit":
                unit_id=element.attrib.get("id")
                unites[unit_id]=None #valeur par défaut
                for child in element:
                  if child.tag=="type":
                    unites[unit_id]=child.text #on lie l'id de l'unité avec son type
            elif element.tag=="relation":
                relation_id=element.attrib.get("id")
                relations_data={
                    "id":relation_id,
                    "type":None
                }
                for child in element:
                  if child.tag=="term":
                    term=child.attrib.get("id")
            else:
              pass

        for unit in unites:
            for type in relation_data:
                if unit==term: #si un id de unit et un unit de term match, on affiche une relation entre les deux dans le html
                  pass #? comment faire

        #on génère le html
        html_content="""
        <!DOCTYPE html>
        <html lang="fr">
        <head>
            <meta charset="UTF-8">
            <title>Annotations Glozz</title>
            <style>
                body {
                    font-family:Arial,sans-serif;
                    line-height:1.8;
                    background:#eaeaea;
                    padding:20px;
                }
                .unit {
                    color:white;
                    background-color:blue;
                    padding:2px 5px;
                    border-radius:4px;
                }
                .relation{
                    color: green;
                    font-style:italic;
                }
                .arrow{
                    display:inline-block;
                    margin:5px;
                    font-size:16px;
                }
            </style>
        </head>
        <body>
            <h1 style="text-align: center;">Texte annoté</h1>
            <div style="background: #fff; padding: 20px; border: 1px solid #ddd; border-radius: 8px;">
        """

        #on ajoute les unités
        html_content+="<h2>Unités</h2><ul>"
        for unit_id,unit_type in unites.items():
            html_content+=f'<li class="unit" title="{unit_type}">Unité {unit_id} : {unit_type}</li>'
        html_content+="</ul>"

        #et les relations
        html_content+="<h2>Relations</h2><ul>"
        for relation in relations:
            source=relation["source"]
            rel_type=relation["type"]
            html_content+=f"""
            <li class="relation">
                <span>Relation {relation['id']}: </span>
                <span class="arrow">→</span>
                (<em>{rel_type}</em>)
            </li>
            """
        html_content+="</ul>"

        html_content+=f"<h2>Texte Brut</h2><p>{texte_brut}</p>"

        #fermeture des balises HTML
        html_content+="""
            </div>
        </body>
        </html>
        """

        #on enregistre dans un fichier html
        output_file=self.nom_fichier.replace(".ac","_annotations.html")
        with open(output_file,"w",encoding="utf-8") as f:
            f.write(html_content)

        print(f"Annotations sauvegardées dans {output_file}")

##Test sur un fichier txt

In [ ]:
if __name__=="__main__":
    lorem=Txt('lorem.txt')
    print("Texte sans annotations : ")
    texte_brut=lorem.afficher_texte_brut()
    print(texte_brut)

    print(f"Nombre de caractères : {lorem.compter_caracteres()}")
    print(f"Nombre de mots : {lorem.compter_mots()}")

##Tests sur des fichiers WebAnno

###Fichier EC-CE1-2015-S47_doc000001_webanno.tsv (annotations POS)

In [ ]:
if __name__=="__main__":
    copie_ce1=WebAnno('EC-CE1-2015-S47_doc000001_webanno.tsv')

    print("Texte extrait sans annotations :")
    texte_brut=copie_ce1.afficher_texte_brut()
    print(texte_brut)

    print(f"Nombre de caractères : {copie_ce1.compter_caracteres()}")
    print(f"Nombre de mots : {copie_ce1.compter_mots()}")

    copie_ce1.convertir('txt')

    copie_ce1.afficher_annotations()

###Fichier essai.tsv (annotations coréférences)

In [ ]:
if __name__=="__main__":
    test=WebAnno('essai.tsv')

    print("Texte extrait sans annotations :")
    texte_brut=test.afficher_texte_brut()
    #print(texte_brut)

    print(f"Nombre de caractères : {test.compter_caracteres()}")
    print(f"Nombre de mots : {test.compter_mots()}")

    test.convertir('txt')

    test.afficher_annotations()

##Test sur des fichiers Glozz

In [ ]:
glozz_fichier=Glozz('glozz_txt_test.ac','glozz_tags_test.aam','glozz_annot_test.aa')

#Lecture et affichage du texte brut
texte_brut=glozz_fichier.afficher_texte_brut()
print("Texte brut : ")
print(texte_brut)

glozz_fichier.convertir('txt')

print(f"Nombre de caractères : {glozz_fichier.compter_caracteres()}")
print(f"Nombre de mots : {glozz_fichier.compter_mots()}")

#affichage des annotations
glozz_fichier.afficher_annotations()